<a href="https://colab.research.google.com/github/s34836/WUM/blob/main/Lab_03_Gradient_Descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 03 - Gradient Descent

## Task 1

Determine the derivative of the function $f(\mathbf{x})=2x_1^2-4x_1 + x_2^2 - x_2 + 2x_3^2 + 4$, then perform one step of gradient descent starting with $\mathbf{x_0}=(0, 1, 2)$.

$$ \mathbf{x}_n = \mathbf{x}_{n-1} - \alpha \nabla f(\mathbf{x_{n-1}}) $$

Przyjmij $\alpha = 0.2$.

## Task 2

Tensorflow has built in automatic differentiation capability, which we can use to calculate the gradients for arbitrarily complex operations


In [ ]:
import tensorflow as tf

def f(x1, x2):
    return 2. * tf.square(x1) + 3. * x1 + 4. * x2 + 5.

x1 = tf.Variable(3.)
x2 = tf.Variable(4.)

with tf.GradientTape() as tape:
    y = f(x1, x2)

dy_dx1, dy_dx2 = tape.gradient(y, [x1, x2])
print(float(dy_dx1), float(dy_dx2))

15.0 4.0


Apply the chain rule to determine $\frac{\partial y}{\partial x_1}$ and $\frac{\partial y}{\partial x_2}$ for:
1. $y = \sigma(f(x_1, x_2))$, where $f(x_1, x_2)= 2 x_1 + 3 x_2$ i $\sigma(x)=\frac{e^x}{1 + e^x}$,
2. $y = g(f(x_1, x_2))$, where $f(x_1, x_2)= x_1^2 + 2 x_2$ i $g(x)=\sin x$.

Next, calculate the derivatives using `GradientTape` and compare the results.

In [1]:
import tensorflow as tf
import numpy as np
def sigma(x):
    return 1. / (1. + tf.exp(-x))

def f(x1, x2):
    return 2. * x1 + 3. * x2

def F(x1, x2):
    return sigma(f(x1, x2))

def F_prime(x1, x2):
    dy_dx1 = 2. * sigma(f(x1, x2)) * (1 - sigma(f(x1, x2)))
    dy_dx2 = 3. * sigma(f(x1, x2)) * (1 - sigma(f(x1, x2)))
    return float(dy_dx1), float(dy_dx2)

def F_prime_tf(x1, x2):
    with tf.GradientTape() as tape:
        y = F(x1, x2)

    dy_dx1, dy_dx2 = tape.gradient(y, [x1, x2])
    return float(dy_dx1), float(dy_dx2)

x1 = tf.Variable(-2.)
x2 = tf.Variable(1.)

print(F_prime(x1, x2))
print(F_prime_tf(x1, x2))


(0.3932238817214966, 0.5898358821868896)
(0.3932238817214966, 0.5898358225822449)


In [3]:
import tensorflow as tf
import numpy as np
def sin(x):
    return sin(x)

def f(x1, x2):
    return tf.square(x1) + 2. * x2

def F(x1, x2):
    return tf.math.sin(f(x1, x2))

def F_prime(x1, x2):
    dy_dx1 = 2. * x1 * tf.math.cos(f(x1, x2))
    dy_dx2 = 2. * tf.math.cos(f(x1, x2))
    return float(dy_dx1), float(dy_dx2)

def F_prime_tf(x1, x2):
    with tf.GradientTape() as tape:
        y = F(x1, x2)

    dy_dx1, dy_dx2 = tape.gradient(y, [x1, x2])
    return float(dy_dx1), float(dy_dx2)

x1 = tf.Variable(-2.)
x2 = tf.Variable(1.)

print(F_prime(x1, x2))
print(F_prime_tf(x1, x2))

(-3.8406810760498047, 1.9203405380249023)
(-3.8406810760498047, 1.9203405380249023)


## Task 3

1.
1. Modify the perceptron implementation from Lab 1 to use the sigmoid activation function.
2. Modify the perceptron implementation below to use `GradientTape` instead of manually derived formulas to calculate gradients.

In [ ]:
import numpy as np
import tensorflow as tf

def sigma(z):
    return 1. / (1. + tf.exp(-z))

class Perceptron:
    def __init__(self, learning_rate=0.01, epochs=100):
        self.learning_rate = learning_rate
        self.epochs = epochs

    def predict(self, X):
        Z = tf.linalg.matvec(X, self.weights) + self.bias
        return sigma(Z)

    def fit(self, X, y):
        self.weights = tf.Variable(np.random.rand(X.shape[1]), dtype=tf.float32)
        self.bias = tf.Variable(np.random.rand(), dtype=tf.float32)

        X = tf.constant(X, dtype=tf.float32)

        for _ in range(self.epochs):
            y_pred = self.predict(X)
            loss = tf.reduce_sum(tf.square(y - y_pred))

            w_gradient = -tf.linalg.matvec(tf.transpose(X), (y - y_pred) * y_pred * (1. - y_pred))
            b_gradient = -tf.reduce_sum((y - y_pred) * y_pred * (1. - y_pred))

            self.weights.assign_sub(self.learning_rate * w_gradient)
            self.bias.assign_sub(self.learning_rate * b_gradient)

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import scale

X, y = load_breast_cancer(return_X_y=True)
X = scale(X)

X = tf.constant(X, dtype=tf.float32)

perceptron = Perceptron(learning_rate=0.01, epochs=100)
perceptron.fit(X, y)
y_pred = perceptron.predict(X) >= 0.5
print(np.mean(y_pred == y))

0.9859402460456942
